## Import Library

In [140]:
import pandas as pd
import re

## Load Dataset

In [ ]:
skills = pd.read_csv("../data/raw/archive/job_skills.csv")
summary = pd.read_csv("../data/raw/archive/job_summary.csv")
posting = pd.read_csv("../data/raw/archive/linkedin_job_postings.csv")

# Merge Dataset
df = (
    skills
    .merge(summary, on="job_link", how="left")
    .merge(posting, on="job_link", how="left")
)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

## Group Column yang diperlukan saja

In [ ]:
columns_to_keep = [
    "job_title", 
    "job_skills", 
    "job_summary", 
    "job_type", 
    "job_level",
    "company"
]

df = df[columns_to_keep]

In [ ]:
df.head()

## Save Dataset yang sudah di butuhkan

In [ ]:
# df.to_csv("../data/raw/Small Dataset/small_dataset.csv", index=False)

## Panggil Dataset yang sudah di save sebelumnya

In [141]:
# Call Directory file csv
df_updated = pd.read_csv("../data/raw/Small Dataset/small_dataset.csv")

In [142]:
# Data Setelah Drop Column
df_updated.head()

,job_title,job_skills,job_summary,job_type,job_level,company
0,Housekeeper I - PT,"Building Custodial Services, Cleaning, Janitor...",Department:\nBuilding Custodial Services\nSala...,Onsite,Mid senior,Jacksonville State University
1,Assistant General Manager - Huntington 4131,"Customer service, Restaurant management, Food ...",Summary Of Key Responsibilities\nThis position...,Onsite,Mid senior,Ruby Tuesday
2,School-based Behavior Analyst,"Applied Behavior Analysis (ABA), Data analysis...",Make a difference every day by joining CCRES a...,Onsite,Mid senior,CCRES Educational and Behavioral Health Services
3,Electrical Deputy Engineering Group Supervisor,"Electrical Engineering, Project Controls, Sche...",Requisition ID: 271524\nRelocation Authorized:...,Onsite,Mid senior,Energy Jobline
4,Electrical Assembly Lead,"Electrical Assembly, Point to point wiring, St...",Job Description\nProduction Specialist\nElectr...,Onsite,Mid senior,Sanmina


In [143]:
# Cek info data
df_updated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296381 entries, 0 to 1296380
Data columns (total 6 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   job_title    1296381 non-null  object
 1   job_skills   1294296 non-null  object
 2   job_summary  1296381 non-null  object
 3   job_type     1296381 non-null  object
 4   job_level    1296381 non-null  object
 5   company      1296372 non-null  object
dtypes: object(6)
memory usage: 59.3+ MB


In [144]:
# Cek Missing Values
df_updated.isnull().sum()

job_title         0
job_skills     2085
job_summary       0
job_type          0
job_level         0
company           9
dtype: int64

## Panggil Dataset Resume

In [ ]:
df_res = pd.read_csv("../data/raw/Resume/resume.csv")

In [146]:
df_res.head()

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [147]:
df_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2484 entries, 0 to 2483
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           2484 non-null   int64 
 1   Resume_str   2484 non-null   object
 2   Resume_html  2484 non-null   object
 3   Category     2484 non-null   object
dtypes: int64(1), object(3)
memory usage: 77.8+ KB


In [148]:
df_res.isnull().sum()

ID             0
Resume_str     0
Resume_html    0
Category       0
dtype: int64

In [149]:
# Tambahkan kolom agar sama dengan dataset utama
df_res["job_skills"] = ""           # resume tidak punya skill list
df_res["job_type"] = "resume"       # tandai sumber
df_res["job_level"] = "resume"
df_res["company"] = "Unknown"

In [150]:
df_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2484 entries, 0 to 2483
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           2484 non-null   int64 
 1   Resume_str   2484 non-null   object
 2   Resume_html  2484 non-null   object
 3   Category     2484 non-null   object
 4   job_skills   2484 non-null   object
 5   job_type     2484 non-null   object
 6   job_level    2484 non-null   object
 7   company      2484 non-null   object
dtypes: int64(1), object(7)
memory usage: 155.4+ KB


In [151]:
columns_to_keep = [
    "Category",
    "job_skills",
    "Resume_str",
    "job_type",
    "job_level",
    "company"
]

df_res = df_res[columns_to_keep]

In [152]:
df_res.tail()

,Category,job_skills,Resume_str,job_type,job_level,company
2479,AVIATION,,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,resume,resume,Unknown
2480,AVIATION,,"GOVERNMENT RELATIONS, COMMUNICATIONS ...",resume,resume,Unknown
2481,AVIATION,,GEEK SQUAD AGENT Professional...,resume,resume,Unknown
2482,AVIATION,,PROGRAM DIRECTOR / OFFICE MANAGER ...,resume,resume,Unknown
2483,AVIATION,,STOREKEEPER II Professional Sum...,resume,resume,Unknown


## Cleaning Category & Resume untuk dataset Resume

In [153]:
def clean_title(t):
    t = t.lower().strip()
    t = re.sub(r"[^a-z0-9+ ]", " ", t)
    t = re.sub(r"\s+", " ", t)
    return t.strip()

In [154]:
df_res["Category"] = df_res["Category"].apply(clean_title)

In [155]:
df_res.head()

,Category,job_skills,Resume_str,job_type,job_level,company
0,hr,,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,resume,resume,Unknown
1,hr,,"HR SPECIALIST, US HR OPERATIONS ...",resume,resume,Unknown
2,hr,,HR DIRECTOR Summary Over 2...,resume,resume,Unknown
3,hr,,HR SPECIALIST Summary Dedica...,resume,resume,Unknown
4,hr,,HR MANAGER Skill Highlights ...,resume,resume,Unknown


In [156]:
RESUME_STOPWORDS = [
    "urgent", "hiring", "opportunity", "apply", "apply now", 
    "join our team", "we are hiring", "immediate", "looking for",
    "job description", "job summary"
]

In [157]:
def clean_resume(text):
    t = text.lower()
    for sw in RESUME_STOPWORDS:
        t = t.replace(sw, " ")
    t = re.sub(r"[^a-z0-9 ]", " ", t)
    t = re.sub(r"\s+", " ", t)
    return t.strip()

df_res["Resume_str"] = df_res["Resume_str"].apply(clean_resume)

In [158]:
df_res.head()

,Category,job_skills,Resume_str,job_type,job_level,company
0,hr,,hr administrator marketing associate hr admini...,resume,resume,Unknown
1,hr,,hr specialist us hr operations summary versati...,resume,resume,Unknown
2,hr,,hr director summary over 20 years experience i...,resume,resume,Unknown
3,hr,,hr specialist summary dedicated driven and dyn...,resume,resume,Unknown
4,hr,,hr manager skill highlights hr skills hr depar...,resume,resume,Unknown


## Rename column dataset Resume

In [159]:
df_res = df_res.rename(columns={"Category": "job_title", "Resume_str": "job_summary"})

In [160]:
df_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2484 entries, 0 to 2483
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   job_title    2484 non-null   object
 1   job_skills   2484 non-null   object
 2   job_summary  2484 non-null   object
 3   job_type     2484 non-null   object
 4   job_level    2484 non-null   object
 5   company      2484 non-null   object
dtypes: object(6)
memory usage: 116.6+ KB


In [161]:
mask = df_res['job_title'].str.contains('hr', case=False, na=False)

total = mask.sum()
print("Total kemunculan HR:", total)

Total kemunculan HR: 110


## Merge dataset utama dengan dataset resume

In [162]:
df_updated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296381 entries, 0 to 1296380
Data columns (total 6 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   job_title    1296381 non-null  object
 1   job_skills   1294296 non-null  object
 2   job_summary  1296381 non-null  object
 3   job_type     1296381 non-null  object
 4   job_level    1296381 non-null  object
 5   company      1296372 non-null  object
dtypes: object(6)
memory usage: 59.3+ MB


In [163]:
df_main = pd.concat([df_updated, df_res], ignore_index=True)

In [164]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1298865 entries, 0 to 1298864
Data columns (total 6 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   job_title    1298865 non-null  object
 1   job_skills   1296780 non-null  object
 2   job_summary  1298865 non-null  object
 3   job_type     1298865 non-null  object
 4   job_level    1298865 non-null  object
 5   company      1298856 non-null  object
dtypes: object(6)
memory usage: 59.5+ MB


## Handling Missing Values

In [165]:
df_main.head()

,job_title,job_skills,job_summary,job_type,job_level,company
0,Housekeeper I - PT,"Building Custodial Services, Cleaning, Janitor...",Department:\nBuilding Custodial Services\nSala...,Onsite,Mid senior,Jacksonville State University
1,Assistant General Manager - Huntington 4131,"Customer service, Restaurant management, Food ...",Summary Of Key Responsibilities\nThis position...,Onsite,Mid senior,Ruby Tuesday
2,School-based Behavior Analyst,"Applied Behavior Analysis (ABA), Data analysis...",Make a difference every day by joining CCRES a...,Onsite,Mid senior,CCRES Educational and Behavioral Health Services
3,Electrical Deputy Engineering Group Supervisor,"Electrical Engineering, Project Controls, Sche...",Requisition ID: 271524\nRelocation Authorized:...,Onsite,Mid senior,Energy Jobline
4,Electrical Assembly Lead,"Electrical Assembly, Point to point wiring, St...",Job Description\nProduction Specialist\nElectr...,Onsite,Mid senior,Sanmina


In [166]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1298865 entries, 0 to 1298864
Data columns (total 6 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   job_title    1298865 non-null  object
 1   job_skills   1296780 non-null  object
 2   job_summary  1298865 non-null  object
 3   job_type     1298865 non-null  object
 4   job_level    1298865 non-null  object
 5   company      1298856 non-null  object
dtypes: object(6)
memory usage: 59.5+ MB


In [167]:
df_main.isnull().sum()

job_title         0
job_skills     2085
job_summary       0
job_type          0
job_level         0
company           9
dtype: int64

In [172]:
# Handling Missing Values untuk Column job_skills
df_main['job_skills'] = df_main['job_skills'].fillna('')

# Handling Missing Values untuk Column company
df_main["company"] = df_main["company"].fillna("Unknown")

In [173]:
df_main.head()

,job_title,job_skills,job_summary,job_type,job_level,company
0,Housekeeper I - PT,"Building Custodial Services, Cleaning, Janitor...",Department:\nBuilding Custodial Services\nSala...,Onsite,Mid senior,Jacksonville State University
1,Assistant General Manager - Huntington 4131,"Customer service, Restaurant management, Food ...",Summary Of Key Responsibilities\nThis position...,Onsite,Mid senior,Ruby Tuesday
2,School-based Behavior Analyst,"Applied Behavior Analysis (ABA), Data analysis...",Make a difference every day by joining CCRES a...,Onsite,Mid senior,CCRES Educational and Behavioral Health Services
3,Electrical Deputy Engineering Group Supervisor,"Electrical Engineering, Project Controls, Sche...",Requisition ID: 271524\nRelocation Authorized:...,Onsite,Mid senior,Energy Jobline
4,Electrical Assembly Lead,"Electrical Assembly, Point to point wiring, St...",Job Description\nProduction Specialist\nElectr...,Onsite,Mid senior,Sanmina


In [174]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1298865 entries, 0 to 1298864
Data columns (total 6 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   job_title    1298865 non-null  object
 1   job_skills   1298865 non-null  object
 2   job_summary  1298865 non-null  object
 3   job_type     1298865 non-null  object
 4   job_level    1298865 non-null  object
 5   company      1298865 non-null  object
dtypes: object(6)
memory usage: 59.5+ MB


In [175]:
df_main.isnull().sum()

job_title      0
job_skills     0
job_summary    0
job_type       0
job_level      0
company        0
dtype: int64

## Clean "job_title"

In [176]:
def clean_title(t):
    t = t.lower().strip()
    
    # hapus simbol yang aneh
    t = re.sub(r"[^a-z0-9+ ]", " ", t)

    # rapikan spasi
    t = re.sub(r"\s+", " ", t)

    return t.strip()

df_main["job_title_clean"] = df_main["job_title"].apply(clean_title)

In [177]:
df_main.head()

,job_title,job_skills,job_summary,job_type,job_level,company,job_title_clean
0,Housekeeper I - PT,"Building Custodial Services, Cleaning, Janitor...",Department:\nBuilding Custodial Services\nSala...,Onsite,Mid senior,Jacksonville State University,housekeeper i pt
1,Assistant General Manager - Huntington 4131,"Customer service, Restaurant management, Food ...",Summary Of Key Responsibilities\nThis position...,Onsite,Mid senior,Ruby Tuesday,assistant general manager huntington 4131
2,School-based Behavior Analyst,"Applied Behavior Analysis (ABA), Data analysis...",Make a difference every day by joining CCRES a...,Onsite,Mid senior,CCRES Educational and Behavioral Health Services,school based behavior analyst
3,Electrical Deputy Engineering Group Supervisor,"Electrical Engineering, Project Controls, Sche...",Requisition ID: 271524\nRelocation Authorized:...,Onsite,Mid senior,Energy Jobline,electrical deputy engineering group supervisor
4,Electrical Assembly Lead,"Electrical Assembly, Point to point wiring, St...",Job Description\nProduction Specialist\nElectr...,Onsite,Mid senior,Sanmina,electrical assembly lead


## Stopword

In [178]:
JOB_STOPWORDS = [
    "urgent", "hiring", "opportunity", "apply", "apply now", 
    "join our team", "we are hiring", "immediate", "looking for",
    "job description", "job summary"
]

In [179]:
def clean_summary(text):
    t = text.lower()

    # hapus stopwords khusus lowongan kerja
    for sw in JOB_STOPWORDS:
        t = t.replace(sw, " ")

    # hilangkan simbol
    t = re.sub(r"[^a-z0-9 ]", " ", t)

    # rapikan spasi
    t = re.sub(r"\s+", " ", t)

    return t.strip()

df_main["job_summary_clean"] = df_main["job_summary"].apply(clean_summary)

In [180]:
df_main.head()

,job_title,job_skills,job_summary,job_type,job_level,company,job_title_clean,job_summary_clean
0,Housekeeper I - PT,"Building Custodial Services, Cleaning, Janitor...",Department:\nBuilding Custodial Services\nSala...,Onsite,Mid senior,Jacksonville State University,housekeeper i pt,department building custodial services salary ...
1,Assistant General Manager - Huntington 4131,"Customer service, Restaurant management, Food ...",Summary Of Key Responsibilities\nThis position...,Onsite,Mid senior,Ruby Tuesday,assistant general manager huntington 4131,summary of key responsibilities this position ...
2,School-based Behavior Analyst,"Applied Behavior Analysis (ABA), Data analysis...",Make a difference every day by joining CCRES a...,Onsite,Mid senior,CCRES Educational and Behavioral Health Services,school based behavior analyst,make a difference every day by joining ccres a...
3,Electrical Deputy Engineering Group Supervisor,"Electrical Engineering, Project Controls, Sche...",Requisition ID: 271524\nRelocation Authorized:...,Onsite,Mid senior,Energy Jobline,electrical deputy engineering group supervisor,requisition id 271524 relocation authorized na...
4,Electrical Assembly Lead,"Electrical Assembly, Point to point wiring, St...",Job Description\nProduction Specialist\nElectr...,Onsite,Mid senior,Sanmina,electrical assembly lead,production specialist electrical wiring assemb...


In [181]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1298865 entries, 0 to 1298864
Data columns (total 8 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   job_title          1298865 non-null  object
 1   job_skills         1298865 non-null  object
 2   job_summary        1298865 non-null  object
 3   job_type           1298865 non-null  object
 4   job_level          1298865 non-null  object
 5   company            1298865 non-null  object
 6   job_title_clean    1298865 non-null  object
 7   job_summary_clean  1298865 non-null  object
dtypes: object(8)
memory usage: 79.3+ MB


## Parser

In [183]:
def parse_skills(x):
    if pd.isna(x) or not isinstance(x, str) or x.strip() == "":
        return []

    # Lowercase + strip
    x = x.lower().strip()

    # Hapus bracket dan quotes
    x = x.replace("[", "").replace("]", "")
    x = x.replace("'", "").replace('"', "")

    # Hapus bullet, dash, titik, simbol aneh
    x = re.sub(r"[•\-–—*]", " ", x)

    # Split berdasarkan koma, pipe, atau semicolon
    parts = re.split(r"[,;|]", x)

    skills = []
    for p in parts:
        p = p.strip()
        if not p:
            continue
        
        # Replace multi space → single space
        p = re.sub(r"\s+", " ", p)

        # Replace space → underscore
        p = p.replace(" ", "_")

        skills.append(p)

    # Hilangkan duplikat sambil mempertahankan urutan
    skills = list(dict.fromkeys(skills))

    return skills

df_main["skills_token"] = df_main["job_skills"].apply(parse_skills)

In [184]:
df_main.head()

,job_title,job_skills,job_summary,job_type,job_level,company,job_title_clean,job_summary_clean,skills_token
0,Housekeeper I - PT,"Building Custodial Services, Cleaning, Janitor...",Department:\nBuilding Custodial Services\nSala...,Onsite,Mid senior,Jacksonville State University,housekeeper i pt,department building custodial services salary ...,"[building_custodial_services, cleaning, janito..."
1,Assistant General Manager - Huntington 4131,"Customer service, Restaurant management, Food ...",Summary Of Key Responsibilities\nThis position...,Onsite,Mid senior,Ruby Tuesday,assistant general manager huntington 4131,summary of key responsibilities this position ...,"[customer_service, restaurant_management, food..."
2,School-based Behavior Analyst,"Applied Behavior Analysis (ABA), Data analysis...",Make a difference every day by joining CCRES a...,Onsite,Mid senior,CCRES Educational and Behavioral Health Services,school based behavior analyst,make a difference every day by joining ccres a...,"[applied_behavior_analysis_(aba), data_analysi..."
3,Electrical Deputy Engineering Group Supervisor,"Electrical Engineering, Project Controls, Sche...",Requisition ID: 271524\nRelocation Authorized:...,Onsite,Mid senior,Energy Jobline,electrical deputy engineering group supervisor,requisition id 271524 relocation authorized na...,"[electrical_engineering, project_controls, sch..."
4,Electrical Assembly Lead,"Electrical Assembly, Point to point wiring, St...",Job Description\nProduction Specialist\nElectr...,Onsite,Mid senior,Sanmina,electrical assembly lead,production specialist electrical wiring assemb...,"[electrical_assembly, point_to_point_wiring, s..."


In [185]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1298865 entries, 0 to 1298864
Data columns (total 9 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   job_title          1298865 non-null  object
 1   job_skills         1298865 non-null  object
 2   job_summary        1298865 non-null  object
 3   job_type           1298865 non-null  object
 4   job_level          1298865 non-null  object
 5   company            1298865 non-null  object
 6   job_title_clean    1298865 non-null  object
 7   job_summary_clean  1298865 non-null  object
 8   skills_token       1298865 non-null  object
dtypes: object(9)
memory usage: 89.2+ MB


## Hapus baris tanpa teks apapun

In [186]:
df_main = df_main[
    (df_main["job_title_clean"].str.len() > 0) |
    (df_main["job_summary_clean"].str.len() > 0) |
    (df_main["skills_token"].apply(len) > 0)
]

In [187]:
df_main.head()

,job_title,job_skills,job_summary,job_type,job_level,company,job_title_clean,job_summary_clean,skills_token
0,Housekeeper I - PT,"Building Custodial Services, Cleaning, Janitor...",Department:\nBuilding Custodial Services\nSala...,Onsite,Mid senior,Jacksonville State University,housekeeper i pt,department building custodial services salary ...,"[building_custodial_services, cleaning, janito..."
1,Assistant General Manager - Huntington 4131,"Customer service, Restaurant management, Food ...",Summary Of Key Responsibilities\nThis position...,Onsite,Mid senior,Ruby Tuesday,assistant general manager huntington 4131,summary of key responsibilities this position ...,"[customer_service, restaurant_management, food..."
2,School-based Behavior Analyst,"Applied Behavior Analysis (ABA), Data analysis...",Make a difference every day by joining CCRES a...,Onsite,Mid senior,CCRES Educational and Behavioral Health Services,school based behavior analyst,make a difference every day by joining ccres a...,"[applied_behavior_analysis_(aba), data_analysi..."
3,Electrical Deputy Engineering Group Supervisor,"Electrical Engineering, Project Controls, Sche...",Requisition ID: 271524\nRelocation Authorized:...,Onsite,Mid senior,Energy Jobline,electrical deputy engineering group supervisor,requisition id 271524 relocation authorized na...,"[electrical_engineering, project_controls, sch..."
4,Electrical Assembly Lead,"Electrical Assembly, Point to point wiring, St...",Job Description\nProduction Specialist\nElectr...,Onsite,Mid senior,Sanmina,electrical assembly lead,production specialist electrical wiring assemb...,"[electrical_assembly, point_to_point_wiring, s..."


In [188]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1298865 entries, 0 to 1298864
Data columns (total 9 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   job_title          1298865 non-null  object
 1   job_skills         1298865 non-null  object
 2   job_summary        1298865 non-null  object
 3   job_type           1298865 non-null  object
 4   job_level          1298865 non-null  object
 5   company            1298865 non-null  object
 6   job_title_clean    1298865 non-null  object
 7   job_summary_clean  1298865 non-null  object
 8   skills_token       1298865 non-null  object
dtypes: object(9)
memory usage: 89.2+ MB


## Gabungkan semua fitur menjadi input_text

In [189]:
df_main["input_text"] = (
    df_main["job_title_clean"] + " "
    + df_main["job_summary_clean"] + " "
    + df_main["skills_token"].apply(lambda x: " ".join(x))
).str.strip()

In [190]:
df_main.head()

,job_title,job_skills,job_summary,job_type,job_level,company,job_title_clean,job_summary_clean,skills_token,input_text
0,Housekeeper I - PT,"Building Custodial Services, Cleaning, Janitor...",Department:\nBuilding Custodial Services\nSala...,Onsite,Mid senior,Jacksonville State University,housekeeper i pt,department building custodial services salary ...,"[building_custodial_services, cleaning, janito...",housekeeper i pt department building custodial...
1,Assistant General Manager - Huntington 4131,"Customer service, Restaurant management, Food ...",Summary Of Key Responsibilities\nThis position...,Onsite,Mid senior,Ruby Tuesday,assistant general manager huntington 4131,summary of key responsibilities this position ...,"[customer_service, restaurant_management, food...",assistant general manager huntington 4131 summ...
2,School-based Behavior Analyst,"Applied Behavior Analysis (ABA), Data analysis...",Make a difference every day by joining CCRES a...,Onsite,Mid senior,CCRES Educational and Behavioral Health Services,school based behavior analyst,make a difference every day by joining ccres a...,"[applied_behavior_analysis_(aba), data_analysi...",school based behavior analyst make a differenc...
3,Electrical Deputy Engineering Group Supervisor,"Electrical Engineering, Project Controls, Sche...",Requisition ID: 271524\nRelocation Authorized:...,Onsite,Mid senior,Energy Jobline,electrical deputy engineering group supervisor,requisition id 271524 relocation authorized na...,"[electrical_engineering, project_controls, sch...",electrical deputy engineering group supervisor...
4,Electrical Assembly Lead,"Electrical Assembly, Point to point wiring, St...",Job Description\nProduction Specialist\nElectr...,Onsite,Mid senior,Sanmina,electrical assembly lead,production specialist electrical wiring assemb...,"[electrical_assembly, point_to_point_wiring, s...",electrical assembly lead production specialist...


In [191]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1298865 entries, 0 to 1298864
Data columns (total 10 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   job_title          1298865 non-null  object
 1   job_skills         1298865 non-null  object
 2   job_summary        1298865 non-null  object
 3   job_type           1298865 non-null  object
 4   job_level          1298865 non-null  object
 5   company            1298865 non-null  object
 6   job_title_clean    1298865 non-null  object
 7   job_summary_clean  1298865 non-null  object
 8   skills_token       1298865 non-null  object
 9   input_text         1298865 non-null  object
dtypes: object(10)
memory usage: 99.1+ MB


## Filtering Rare Labels

In [192]:
# Filter labels dengan minimal 200 samples (untuk stabilitas model)
counts = df_main["job_title_clean"].value_counts()
valid_labels = counts[counts >= 200].index
df_main = df_main[df_main["job_title_clean"].isin(valid_labels)]

# Output
print(f"Dataset Statistics:")
print(f"   - Total job titles (classes) : {df_main['job_title_clean'].nunique()}")
print(f"   - Total samples              : {len(df_main)}")
print(f"   - Samples per class (avg)    : {len(df_main) / df_main['job_title_clean'].nunique():.0f}")

Dataset Statistics:
   - Total job titles (classes) : 401
   - Total samples              : 261728
   - Samples per class (avg)    : 653


In [193]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Index: 261728 entries, 7 to 1298306
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   job_title          261728 non-null  object
 1   job_skills         261728 non-null  object
 2   job_summary        261728 non-null  object
 3   job_type           261728 non-null  object
 4   job_level          261728 non-null  object
 5   company            261728 non-null  object
 6   job_title_clean    261728 non-null  object
 7   job_summary_clean  261728 non-null  object
 8   skills_token       261728 non-null  object
 9   input_text         261728 non-null  object
dtypes: object(10)
memory usage: 22.0+ MB


## Remove Noise

In [194]:
df_main = df_main[df_main["input_text"].str.len() > 20]

In [195]:
df_main.head()

,job_title,job_skills,job_summary,job_type,job_level,company,job_title_clean,job_summary_clean,skills_token,input_text
7,Veterinary Receptionist,"Veterinary Receptionist, AAHAaccredited, Custo...",Join the Adventure at\nAustin Bluffs Animal Cl...,Onsite,Mid senior,WellHaven Pet Health,veterinary receptionist,join the adventure at austin bluffs animal cli...,"[veterinary_receptionist, aahaaccredited, cust...",veterinary receptionist join the adventure at ...
13,Travel Allied Health Professional - CT Technol...,"CT Technologist, Allied Health Professional, T...",FlexCare Imaging is seeking a travel CT Techno...,Onsite,Mid senior,FlexCare Medical Staffing,travel allied health professional ct technologist,flexcare imaging is seeking a travel ct techno...,"[ct_technologist, allied_health_professional, ...",travel allied health professional ct technolog...
19,Assistant General Manager,"Hiring, Training, Retaining, Developing Team, ...",Job Description\nAN ASSISTANT GENERAL MANAGER ...,Onsite,Mid senior,The Hill,assistant general manager,an assistant general manager is the agms provi...,"[hiring, training, retaining, developing_team,...",assistant general manager an assistant general...
21,Internal Auditor,"Accounting, Auditing, GAAP, Tax laws, Professi...",Details\nOpen Date\n01/10/2024\nRequisition Nu...,Onsite,Mid senior,University of Utah,internal auditor,details open date 01 10 2024 requisition numbe...,"[accounting, auditing, gaap, tax_laws, profess...",internal auditor details open date 01 10 2024 ...
23,Audit Manager,"Accounting principles, Auditing standards, Fin...",Job Details\nDescription\nKeiter Mission:\nHel...,Onsite,Mid senior,Keiter CPAs,audit manager,job details description keiter mission help pe...,"[accounting_principles, auditing_standards, fi...",audit manager job details description keiter m...


In [196]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Index: 261728 entries, 7 to 1298306
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   job_title          261728 non-null  object
 1   job_skills         261728 non-null  object
 2   job_summary        261728 non-null  object
 3   job_type           261728 non-null  object
 4   job_level          261728 non-null  object
 5   company            261728 non-null  object
 6   job_title_clean    261728 non-null  object
 7   job_summary_clean  261728 non-null  object
 8   skills_token       261728 non-null  object
 9   input_text         261728 non-null  object
dtypes: object(10)
memory usage: 22.0+ MB


## Cek Duplicates

In [197]:
total_duplicates = df_main["input_text"].duplicated().sum()
print(f"Duplicate Statistics:")
print(f"   - Total duplicates             : {total_duplicates}")
print(f"   - Percentage                   : {(total_duplicates/len(df_main)*100):.2f}%")

Duplicate Statistics:
   - Total duplicates             : 1569
   - Percentage                   : 0.60%


## Handling Duplicates

In [199]:
# 1. Pastikan skills_token bisa di-hash
df_main["skills_token"] = df_main["skills_token"].apply(
    lambda x: tuple(x) if isinstance(x, list) else x
)

# 2. Drop duplicates pada 4 kolom
dedup_cols = ["job_title_clean", "job_summary_clean", "skills_token", "input_text"]

df_main = df_main.drop_duplicates(subset=dedup_cols, keep="first")

# 3. Reset index
df_main = df_main.reset_index(drop=True)

print("Done deduplication!")
print("Final shape : ", df_main.shape)

Done deduplication!
Final shape :  (260159, 10)


In [200]:
# Ubah regex
df_main["skills_token"] = df_main["skills_token"].apply(
    lambda x: list(x) if isinstance(x, tuple) else x
)

In [201]:
# Cek Duplicates
total_duplicates = df_main["input_text"].duplicated().sum()
print(f"Duplicate Statistics:")
print(f"   - Total duplicates             : {total_duplicates}")
print(f"   - Percentage                   : {(total_duplicates/len(df_main)*100):.2f}%")

Duplicate Statistics:
   - Total duplicates             : 0
   - Percentage                   : 0.00%


## Drop column yang tidak dibutuhkan

In [203]:
df_main.head()

,job_title,job_skills,job_summary,job_type,job_level,company,job_title_clean,job_summary_clean,skills_token,input_text
0,Veterinary Receptionist,"Veterinary Receptionist, AAHAaccredited, Custo...",Join the Adventure at\nAustin Bluffs Animal Cl...,Onsite,Mid senior,WellHaven Pet Health,veterinary receptionist,join the adventure at austin bluffs animal cli...,"[veterinary_receptionist, aahaaccredited, cust...",veterinary receptionist join the adventure at ...
1,Travel Allied Health Professional - CT Technol...,"CT Technologist, Allied Health Professional, T...",FlexCare Imaging is seeking a travel CT Techno...,Onsite,Mid senior,FlexCare Medical Staffing,travel allied health professional ct technologist,flexcare imaging is seeking a travel ct techno...,"[ct_technologist, allied_health_professional, ...",travel allied health professional ct technolog...
2,Assistant General Manager,"Hiring, Training, Retaining, Developing Team, ...",Job Description\nAN ASSISTANT GENERAL MANAGER ...,Onsite,Mid senior,The Hill,assistant general manager,an assistant general manager is the agms provi...,"[hiring, training, retaining, developing_team,...",assistant general manager an assistant general...
3,Internal Auditor,"Accounting, Auditing, GAAP, Tax laws, Professi...",Details\nOpen Date\n01/10/2024\nRequisition Nu...,Onsite,Mid senior,University of Utah,internal auditor,details open date 01 10 2024 requisition numbe...,"[accounting, auditing, gaap, tax_laws, profess...",internal auditor details open date 01 10 2024 ...
4,Audit Manager,"Accounting principles, Auditing standards, Fin...",Job Details\nDescription\nKeiter Mission:\nHel...,Onsite,Mid senior,Keiter CPAs,audit manager,job details description keiter mission help pe...,"[accounting_principles, auditing_standards, fi...",audit manager job details description keiter m...


In [202]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260159 entries, 0 to 260158
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   job_title          260159 non-null  object
 1   job_skills         260159 non-null  object
 2   job_summary        260159 non-null  object
 3   job_type           260159 non-null  object
 4   job_level          260159 non-null  object
 5   company            260159 non-null  object
 6   job_title_clean    260159 non-null  object
 7   job_summary_clean  260159 non-null  object
 8   skills_token       260159 non-null  object
 9   input_text         260159 non-null  object
dtypes: object(10)
memory usage: 19.8+ MB


In [204]:
df_main = df_main.drop(columns=[
    "job_title",
    "job_skills",
    "job_summary",
    "job_type",
    "job_level",
    "company"
])

In [205]:
df_main.head()

,job_title_clean,job_summary_clean,skills_token,input_text
0,veterinary receptionist,join the adventure at austin bluffs animal cli...,"[veterinary_receptionist, aahaaccredited, cust...",veterinary receptionist join the adventure at ...
1,travel allied health professional ct technologist,flexcare imaging is seeking a travel ct techno...,"[ct_technologist, allied_health_professional, ...",travel allied health professional ct technolog...
2,assistant general manager,an assistant general manager is the agms provi...,"[hiring, training, retaining, developing_team,...",assistant general manager an assistant general...
3,internal auditor,details open date 01 10 2024 requisition numbe...,"[accounting, auditing, gaap, tax_laws, profess...",internal auditor details open date 01 10 2024 ...
4,audit manager,job details description keiter mission help pe...,"[accounting_principles, auditing_standards, fi...",audit manager job details description keiter m...


In [206]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260159 entries, 0 to 260158
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   job_title_clean    260159 non-null  object
 1   job_summary_clean  260159 non-null  object
 2   skills_token       260159 non-null  object
 3   input_text         260159 non-null  object
dtypes: object(4)
memory usage: 7.9+ MB


In [207]:
def clean_input_text(text):
    if not isinstance(text, str):
        return ""
    text = text.strip()                         # hilangkan spasi di awal/akhir
    text = re.sub(r"\s+", " ", text)            # normalisasi spasi (hapus spasi berlebih)
    return text

df_main["input_text"] = df_main["input_text"].apply(clean_input_text)

In [208]:
df_main.head()

,job_title_clean,job_summary_clean,skills_token,input_text
0,veterinary receptionist,join the adventure at austin bluffs animal cli...,"[veterinary_receptionist, aahaaccredited, cust...",veterinary receptionist join the adventure at ...
1,travel allied health professional ct technologist,flexcare imaging is seeking a travel ct techno...,"[ct_technologist, allied_health_professional, ...",travel allied health professional ct technolog...
2,assistant general manager,an assistant general manager is the agms provi...,"[hiring, training, retaining, developing_team,...",assistant general manager an assistant general...
3,internal auditor,details open date 01 10 2024 requisition numbe...,"[accounting, auditing, gaap, tax_laws, profess...",internal auditor details open date 01 10 2024 ...
4,audit manager,job details description keiter mission help pe...,"[accounting_principles, auditing_standards, fi...",audit manager job details description keiter m...


In [209]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260159 entries, 0 to 260158
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   job_title_clean    260159 non-null  object
 1   job_summary_clean  260159 non-null  object
 2   skills_token       260159 non-null  object
 3   input_text         260159 non-null  object
dtypes: object(4)
memory usage: 7.9+ MB


In [210]:
df_main.isnull().sum()

job_title_clean      0
job_summary_clean    0
skills_token         0
input_text           0
dtype: int64

## Save clean dataset (Final)

In [212]:
df_main.to_csv("../data/processed/Clean Dataset/cleaned_data.csv", index=False)